In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -uq /content/drive/MyDrive/AdvancedDeepLearning/Project/val.zip -d data
!unzip -uq /content/drive/MyDrive/AdvancedDeepLearning/Project/train.zip -d data
!unzip -uq /content/drive/MyDrive/AdvancedDeepLearning/Project/test.zip -d data

In [72]:
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

train_transform = T.Compose([
  T.RandomResizedCrop(size=256),
  T.ToTensor(),
  T.Normalize(mean_nums, std_nums)
])

test_transform = T.Compose([
  T.RandomResizedCrop(size=256),
  T.ToTensor(),
  T.Normalize(mean_nums, std_nums)
])


val_transform = T.Compose([
  T.RandomResizedCrop(size=256),
  T.ToTensor(),
  T.Normalize(mean_nums, std_nums)
])


train_dataset = ImageFolder('data/train', train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = ImageFolder('data/test', test_transform)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True)

val_dataset = ImageFolder('data/val', val_transform)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)


In [57]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 3, 256, 256])
Labels batch shape: torch.Size([32])


In [58]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [59]:
def create_model(n_classes):
  model = models.resnet18(pretrained=True)

  n_features = model.fc.in_features
  model.fc = nn.Linear(n_features, n_classes)

  return model.to(device)

In [60]:
from torchvision import models
from torch import nn, optim
model = create_model(1)

In [61]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for inputs, labels in data_loader:
    inputs = inputs.to(device)
    labels = labels.unsqueeze(1).to(device).float()

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = loss_fn(outputs, labels)
    losses.append(loss.item())
    loss.backward()
    optimizer.step()

    y_pred = torch.round(torch.sigmoid(outputs))
    correct_predictions += torch.sum(y_pred == labels)


  return correct_predictions.double() / n_examples, np.mean(losses)

In [62]:
from torch import optim
import numpy as np


optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_fn = nn.BCEWithLogitsLoss()

n_epochs = 10

for epoch in range(n_epochs):
  print(f'Epoch {epoch + 1}/{n_epochs}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device, len(train_dataset))
  print(f'Train loss {train_loss} accuracy {train_acc}')


Epoch 1/10
----------
Train loss 0.2787023243134373 accuracy 0.875
Epoch 2/10
----------
Train loss 0.1675297680762282 accuracy 0.9392254601226994
Epoch 3/10
----------
Train loss 0.14752612048131555 accuracy 0.942101226993865
Epoch 4/10
----------
Train loss 0.13971481753364662 accuracy 0.946319018404908
Epoch 5/10
----------
Train loss 0.13315761287016736 accuracy 0.9522622699386504
Epoch 6/10
----------
Train loss 0.11855299486293018 accuracy 0.9539877300613497
Epoch 7/10
----------
Train loss 0.11591820121725645 accuracy 0.9516871165644172
Epoch 8/10
----------
Train loss 0.10543542252167877 accuracy 0.95954754601227
Epoch 9/10
----------
Train loss 0.10933339977465524 accuracy 0.9570552147239264
Epoch 10/10
----------
Train loss 0.09697776392353641 accuracy 0.9629984662576687


In [88]:
def validate(net, loader):
  #device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  print("testing network:")
  correct = 0
  total = 0
  with torch.no_grad():
    for inputs, labels in loader:
      inputs = inputs.to(device)
      labels = labels.unsqueeze(1).to(device)
      
      outputs = net(inputs)
      y_pred = torch.round(torch.sigmoid(outputs))
      #correct += torch.sum(y_pred == labels)
      correct += (y_pred == labels).sum().float()
      total += labels.size(0)
    print("total accuracy of net: %.2f%%" % (correct/total*100))

In [89]:
model = model.eval()
validate(model, test_loader)

testing network:
total accuracy of net: 94.07%


In [91]:
validate(model, val_loader)

testing network:
total accuracy of net: 81.25%
